# Setting 

In [1]:
import logging
import os
import pdb
import math
import glob
import random
import time
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.preprocessing import LabelEncoder

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

import torchvision.transforms as T

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.nn.parameter import Parameter

from tqdm import tqdm
import sys

import copy
from scipy import spatial
import csv

from perceiver import crop, patchify, get_patch_coords, ImageDataset, PerceiverBlock, Perceiver, CustomDataset, CombinedModel

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

# Load model, trainset

## Load Model Diff Result

In [28]:
사용자_path = '/home/Minju/Perceiver/' # 계정에 따라 이 부분만 바꿔주면 됩니다.
data_path = 사용자_path + 'code/'
model_path = 사용자_path + 'model/'
loader_dir = 사용자_path + 'loader/'

In [9]:
cos_diff = pd.read_csv(data_path + 'best_cosine_results.txt', sep="\t")
euc_diff = pd.read_csv(data_path + 'best_euclidean_results.txt', sep="\t")

In [10]:
cos_diff

,Task_ID,Best_Target_ID,Best_Cos_Score,Task_Modality,Target_Modality
0,0,7,0.495981,Text,Image
1,1,8,0.491464,Text,Image
2,2,11,0.513187,Text,Image
3,3,11,0.536841,Text,Image
4,4,6,0.538693,Text,Image
5,5,8,0.463215,Text,Image
6,6,11,0.650865,Image,Image
7,7,11,0.649983,Image,Image
8,8,6,0.574057,Image,Image
9,9,11,0.517529,Image,Image


In [11]:
euc_diff

,Task_ID,Best_Target_ID,Best_Euc_Score,Task_Modality,Target_Modality
0,0,7,0.361935,Text,Image
1,1,8,0.343263,Text,Image
2,2,11,0.321216,Text,Image
3,3,11,0.427299,Text,Image
4,4,6,0.409382,Text,Image
5,5,8,0.316713,Text,Image
6,6,11,0.535227,Image,Image
7,7,11,0.377393,Image,Image
8,8,6,0.428867,Image,Image
9,9,6,0.414685,Image,Image


In [30]:
input_models = []
for i in range (6):
    text_model = torch.load(model_path + f'text_model_{i+1}.pkl')
    input_models.append(text_model)
    print(f"Text model {i+1}번 불러오기 완료.")

/tmp/ipykernel_281802/2228528905.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_model = torch.load(model_path + f'text_model_{i+1}.pkl')


Text model 1번 불러오기 완료.
Text model 2번 불러오기 완료.
Text model 3번 불러오기 완료.
Text model 4번 불러오기 완료.
Text model 5번 불러오기 완료.
Text model 6번 불러오기 완료.


In [31]:
for i in range(6):
    img_model = torch.load(model_path + f'image_model_{i+1}.pkl')
    input_models.append(img_model)
    print(f"Image model {i}번 불러오기 완료.")

/tmp/ipykernel_281802/1837349535.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  img_model = torch.load(model_path + f'image_model_{i+1}.pkl')


Image model 0번 불러오기 완료.
Image model 1번 불러오기 완료.
Image model 2번 불러오기 완료.
Image model 3번 불러오기 완료.
Image model 4번 불러오기 완료.
Image model 5번 불러오기 완료.


## Target의 Backbone 설정

In [32]:
target_id = 0 
batch_size = 32

In [33]:
backbone_id = int(euc_diff[euc_diff.Task_ID == target_id].Best_Target_ID.iloc[0])
print(f'Backbone of {target_id} is : {backbone_id}')
target_model = input_models[target_id]
backbone_model = input_models[backbone_id]

Backbone of 0 is : 7


# Knowledge Transfer

## 1. Shared layer info 초기화

## 2. Piggymask 설정

In [ ]:
piggymasks = {}
name='sample'


## 3. 모델 학습 

## 4. Shared layer info로 checkpoint 저장

* 이 부분은 서버 메모리상 일단 나중에 구현